[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/llamaindex/sql-router-query-engine/sql-query-router.ipynb)

## Installations

In [ ]:
%pip install llama_index llama_hub weaviate_client urllib3 llama-cpp-python llama-hub-youtube-transcript llama-index-readers-youtube-transcript

In [1]:
import os
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SQLDatabase,
    download_loader
)
from llama_index.vector_stores.weaviate import WeaviateVectorStore


import weaviate
from weaviate import classes as wvc
import openai

In [2]:

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(
    embed_batch_size=10,
    model="text-embedding-3-small"
)
openai.api_key = os.environ["OPENAI_APIKEY"]
Settings.embed_model = embed_model

## Connect to Weaviate

In [ ]:
client = weaviate.connect_to_embedded(
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]
    }
)

### Create Schema

In [ ]:
client.collections.delete("Podcast")
collection = client.collections.create(
    name="Podcast",
    description="Weaviate podcast",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small"
    ),
    properties=[
        wvc.config.Property(name="content", data_type=wvc.config.DataType.TEXT, description="Content from the podcasts")
    ]
)

## Load in Data

In [25]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()

podcasts = loader.load_data(ytlinks=['https://www.youtube.com/watch?v=xk28RMhRy1U', 'https://www.youtube.com/watch?v=Du6IphCcCec',
'https://www.youtube.com/watch?v=Q7f2JeuMN7E', 'https://www.youtube.com/watch?v=nSCUk5pHXlo'])

In [ ]:
from llama_index.core.node_parser import SimpleNodeParser
print("Documents", len(podcasts))
parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(podcasts)
print("Number of nodes:", len(nodes))
print(nodes[1].metadata)

## Build the Weaviate Index

In [37]:
vector_store = WeaviateVectorStore(weaviate_client=client, index_name="Podcast")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
# ingest data
podcast_index = VectorStoreIndex.from_documents(podcasts, storage_context=storage_context)

In [ ]:
# you can instantiate the index at anytime like this, without ingesting:
podcast_index = WeaviateVectorStore(
    weaviate_client=client, index_name="Podcast"
)

## Create SQL Table

In [43]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [44]:
table_name = "podcast_stats"
podcast_stats_table = Table(
    table_name,
    metadata_obj,
    Column("podcast_title", String(16), primary_key=True),
    Column("views", Integer),
    Column("duration", Integer),
)

metadata_obj.create_all(engine)

In [ ]:
metadata_obj.tables.keys()

In [46]:
from sqlalchemy import insert

rows = [
    {"podcast_title": "Weaviate 1.20", "views": 328, "duration": 65},
    {"podcast_title": "Weaviate 1.19", "views": 280, "duration": 27},
    {"podcast_title": "Weaviate 1.18", "views": 428, "duration": 65},
    {"podcast_title": "Weaviate 1.17", "views": 257, "duration": 43}
]

for row in rows:
  stmt = insert(podcast_stats_table).values(**row)
  with engine.connect() as connection:
    cursor = connection.execute(stmt)
    connection.commit()

## Create SQL Table in LlamaIndex

In [47]:
sql_database = SQLDatabase(engine, include_tables=["podcast_stats"])

In [48]:
from llama_index.core.indices.struct_store.sql_query import NLSQLTableQueryEngine

In [49]:
# set up text2SQL prompt
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["podcast_stats"],
)

In [ ]:
sql_query_engine.query("Which release podcast had the most views?")

## Build Query Engine

In [51]:
vector_query_engine = podcast_index.as_query_engine()

In [ ]:
vector_query_engine.query("Tell me about a cool feature in Weaviate")

## Tell LlamaIndex about the Tools

In [56]:
from llama_index.core.tools.query_engine import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine = sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over a table containing: "
        "podcast_stats, containing the views/duration of each podcast"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for answering semantic questions about Weaviate release podcasts",
)

In [57]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=([sql_tool] + [vector_tool]),
)

## Query

In [ ]:
response = query_engine.query("Which release podcast had the most views?")
print(str(response))

In [ ]:
response = query_engine.query("Tell me about a new feature in Weaviate 1.18")
print(str(response))